In [1]:
import pandas as pd
df = pd.read_csv(filepath_or_buffer='/kaggle/input/data-science-r-ladies-chapter-events/rladies_chapters.csv', index_col=['id'],
                parse_dates=['date'])
# we want to rank the chapters by volume
df['chapter count'] = df['chapter'].apply(lambda x : df['chapter'].value_counts().to_dict()[x])
df.head()

,chapter,title,date,location,year,chapter count
id,,,,,,
296275584,rladies-st-louis,Save the date! Introduction to ChatGPT for R c...,2023-11-30,online,2023,41
296277517,rladies-coventry,An basic introduction: Python for R Users,2023-11-30,online,2023,15
295898711,rladies-baltimore,Holiday graphics and animations in R,2023-11-28,inperson,2023,34
296346610,rladies-philly,TidyTuesday with R-Ladies Philly!,2023-11-14,online,2023,83
296275461,rladies-st-louis,Save the date! Introduction to Network Analysi...,2023-11-08,online,2023,41


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4268 entries, 296275584 to 85320382
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   chapter        4268 non-null   object        
 1   title          4268 non-null   object        
 2   date           4268 non-null   datetime64[ns]
 3   location       4268 non-null   object        
 4   year           4268 non-null   int64         
 5   chapter count  4268 non-null   int64         
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 233.4+ KB


In [3]:
from plotly.express import bar
bar(data_frame=df['location'].value_counts().to_frame().reset_index(), x='location', y='count')

In [4]:
from plotly.express import scatter
scatter(data_frame=df.sort_values(ascending=True, by='chapter count'), x='date', y='chapter', height=3500, hover_name='title', color='year',
       color_continuous_scale='bluered', )

This is probably the nut graf as it ranks the chapters by how many meetings they have had and it shows all the meeting names.

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
from plotly.express import treemap

NGRAM_RANGE = (1, 2)
TOPN = 100
model = CountVectorizer(encoding='utf-8', stop_words='english', min_df=15, max_df=0.07, lowercase=True, ngram_range=NGRAM_RANGE)
model_result = model.fit_transform(raw_documents=df['title'])
title_df = pd.DataFrame(data={'word': model.get_feature_names_out().tolist(), 'count': model_result.toarray().sum(axis=0).tolist()})
title_df['numeric'] = title_df['word'].str.isnumeric()
print(title_df.shape)
treemap(data_frame=title_df[~title_df['numeric']].sort_values(ascending=False, by='count').head(n=TOPN), path=['word'], names='word', values='count', color='count', height=800,
       color_continuous_scale='bluered', title = '{} {}-grams by count'.format(min(len(title_df), TOPN), NGRAM_RANGE))

(288, 3)


We could probably tighten our frequency band a little and find out more about the topics discussed; here is where we learn that some of the meeings were conducted in Spanish. A really good analysis would probably find a way to deal with the Spanish content better.